# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import gc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

/Users/louis/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Load Data

In [2]:
properties_file_path = 'properties_2017.csv' # <--- CHANGE THIS TO THE PATH OF YOUR 'properties_2017.csv'
train_file_path = 'train.csv'

In [3]:
prop = pd.read_csv(properties_file_path, low_memory=False)
prop = prop[prop.fips.notnull()]
train = pd.read_csv(train_file_path, low_memory=False)
original_prop = prop.copy()

# Feature Importance

In [ ]:
prop_columns = pd.DataFrame(columns=['Column Name', 'Data Type', 'NaN Count', 'Unique Values'])
for i,c in enumerate(prop.columns):
    prop_columns.loc[i,'Column Name'] = c
    prop_columns.loc[i,'Data Type'] = prop[c].dtype
    prop_columns.loc[i,'NaN Count'] = prop[c].isnull().sum()    
    prop_columns.loc[i,'Unique Values'] = len(prop[c].unique())

prop_columns

In [ ]:
merged = train.merge(prop,how='left',on='parcelid')
y_train = merged['logerror'].values
merged = merged.drop(['parcelid', 'logerror', 'transactiondate', 'subdate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
merged.fillna(0, inplace=True)

for c in merged.columns:
    if merged[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(merged[c].values))
        merged[c] = lbl.transform(list(merged[c].values))

xgb_params = {
    'eta': 0.05,
    'max_depth': 8,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'silent': 1,
    'seed' : 0
}

dtrain = xgb.DMatrix(merged, y_train, feature_names=merged.columns.values)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=150)

# plot the important features #
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model, height=0.8, ax=ax)
plt.show()

# Null Data Exploration

In [ ]:
null_df = prop.isnull().sum().reset_index()
null_df.columns = ['feature_name', 'missing_count']
null_df.sort_values(by='missing_count', inplace=True)

In [ ]:
ind = np.arange(null_df.shape[0])
fig, ax = plt.subplots(figsize=(10, 20))
rects = ax.barh(ind, null_df.missing_count.values, color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(null_df.feature_name.values, rotation='horizontal')
ax.set_xlabel('Count of null values')
ax.set_title('Number of null values in each column')
plt.show()
plt.gcf().clear()

In [ ]:
null_df['proportion'] = null_df.loc[:,'missing_count'] / len(prop)
null_values_proportion = null_df.sort_values(by='proportion', ascending=False)
print('The following features have more than 97% of their data missing')
print(null_values_proportion[null_values_proportion['proportion'] > 0.97])
drop_list = null_values_proportion[null_values_proportion['proportion'] > 0.97].loc[:,'feature_name'].values
prop.drop(drop_list, 1, inplace=True)

In [ ]:
del null_values_proportion; del drop_list; del null_df; gc.collect()

# Variable Correlation

In [ ]:
nominal_cols = ['airconditioningtypeid','architecturalstyletypeid','buildingqualitytypeid','buildingclasstypeid','decktypeid','fips','hashottuborspa','heatingorsystemtypeid','pooltypeid10','pooltypeid2','pooltypeid7','propertycountylandusecode','propertylandusetypeid','propertyzoningdesc','rawcensustractandblock','regionidcity','regionidcounty','regionidneighborhood','regionidzip','storytypeid','typeconstructiontypeid','yearbuilt','taxdelinquencyflag']
numeric_cols = [col for col in original_prop.columns if col not in nominal_cols]

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(data=original_prop[numeric_cols].corr())
plt.show()
plt.gcf().clear()

In [ ]:
del nominal_cols; del original_prop; del numeric_cols; gc.collect()

In [ ]:
# CORRELATION NOTE
# The following are strongly correlated: 'calculatedfinishedsquarefeet' 'finishedsquarefeet12' 'finishedsquarefeet13' 'finishedsquarefeet15' 'finishedsquarefeet6'
# According to the descriptions they are similiar pieces of information, hence we pick the one with the least missing values and drop the rest

In [4]:
to_exclude = {
    'propertyzoningdesc'
}

# https://www.kaggle.com/c/zillow-prize-1/discussion/40408 - Should not use tax values for predictions
to_exclude = to_exclude | {col for col in prop.columns.values if 'tax' in col}

# getting rid of features with more than 98% missing values
for col in prop.columns:
    num_missing = prop[col].isnull().sum()
    if num_missing == 0:
        continue
    missing_frac = num_missing / float(prop.shape[0])
    if missing_frac > 0.98:
        to_exclude.add(col)
        
for col in prop.columns:
    num_uniques = len(prop[col].unique())
    if prop[col].isnull().sum() != 0:
        num_uniques -= 1
    if num_uniques == 1:
        to_exclude.add(col)

cols = [col for col in prop.columns if col not in to_exclude]
prop = prop.loc[:,cols]
null_df = prop.isnull().sum().reset_index()
null_df.columns = ['feature','null_values']

knn_threshold = 1000000
knn_list = []
for index, row in null_df.iterrows():
    if row['null_values'] < knn_threshold and row['null_values'] > 0:
        knn_list.append(row['feature'])
print(knn_list)

def fill_knn(df, base, target):
    miss = df[target].isnull()
    notmiss = ~miss
    X = df.loc[notmiss, base].values
    Y = df.loc[notmiss, target].values
    if Y.dtype != object:
        print('not str')
        Y = df.loc[notmiss, target].values.astype(int)
    classifier = KNeighborsClassifier()
    print('fitting')
    classifier.fit(X, Y)
    print('predicting')
    predicted = classifier.predict(df.loc[miss, base])
    df.loc[miss, target] = predicted

for item in knn_list:
    print('Filling', item, 'with KNN')
    fill_knn(prop, ['latitude','longitude'], item)
    print('Done')

['bathroomcnt', 'bedroomcnt', 'calculatedbathnbr', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fullbathcnt', 'lotsizesquarefeet', 'propertycountylandusecode', 'regionidcity', 'regionidzip', 'roomcnt', 'yearbuilt', 'assessmentyear', 'censustractandblock']
Filling bathroomcnt with KNN
not str
fitting
predicting
Done
Filling bedroomcnt with KNN
not str
fitting
predicting
Done
Filling calculatedbathnbr with KNN
not str
fitting
predicting
Done
Filling calculatedfinishedsquarefeet with KNN
not str
fitting
predicting
Done
Filling finishedsquarefeet12 with KNN
not str
fitting
predicting
Done
Filling fullbathcnt with KNN
not str
fitting
predicting
Done
Filling lotsizesquarefeet with KNN
not str
fitting
predicting
Done
Filling propertycountylandusecode with KNN
fitting
predicting
Done
Filling regionidcity with KNN
not str
fitting
predicting
Done
Filling regionidzip with KNN
not str
fitting
predicting
Done
Filling roomcnt with KNN
not str
fitting
predicting
Done
Filling yearbuilt wit

In [5]:
prop.fillna(-999, inplace=True)

In [6]:
prop.isnull().sum()

parcelid                        0
airconditioningtypeid           0
bathroomcnt                     0
bedroomcnt                      0
buildingqualitytypeid           0
calculatedbathnbr               0
finishedfloor1squarefeet        0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
finishedsquarefeet15            0
finishedsquarefeet50            0
fips                            0
fireplacecnt                    0
fullbathcnt                     0
garagecarcnt                    0
garagetotalsqft                 0
heatingorsystemtypeid           0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
propertylandusetypeid           0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidneighborhood            0
regionidzip                     0
roomcnt                         0
threequarterbathnbr             0
unitcnt       

In [9]:
prop.to_csv('final_prop_cat.csv', index=False)